In [301]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pandas_ta as ta
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor,MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
import datetime as dt
import pandas_datareader as web
import tensorflow as tf
from tensorflow import keras
import yfinance as yf


### loading the paths

In [340]:
path=pd.read_csv("C:/Users/Thato/Downloads/archive (26)/yield.csv")
yield_df=pd.read_csv('C:/Users/Thato/Downloads/archive (27)/crop_yield.csv')


In [342]:


yield_df

,Region,Soil_Type,Crop,Rainfall_mm,Temperature_Celsius,Fertilizer_Used,Irrigation_Used,Weather_Condition,Days_to_Harvest,Yield_tons_per_hectare
0,West,Sandy,Cotton,897.077239,27.676966,False,True,Cloudy,122,6.555816
1,South,Clay,Rice,992.673282,18.026142,True,True,Rainy,140,8.527341
2,North,Loam,Barley,147.998025,29.794042,False,False,Sunny,106,1.127443
3,North,Sandy,Soybean,986.866331,16.644190,False,True,Rainy,146,6.517573
4,South,Silt,Wheat,730.379174,31.620687,True,True,Cloudy,110,7.248251
...,...,...,...,...,...,...,...,...,...,...
999995,West,Silt,Rice,302.805345,27.987428,False,False,Sunny,76,1.347586
999996,South,Chalky,Barley,932.991383,39.661039,True,False,Rainy,93,7.311594
999997,North,Peaty,Cotton,867.362046,24.370042,True,False,Cloudy,108,5.763182
999998,West,Silt,Wheat,492.812857,33.045505,False,False,Sunny,102,2.070159


In [344]:
yield_df=yield_df.head(10000)
yield_df

,Region,Soil_Type,Crop,Rainfall_mm,Temperature_Celsius,Fertilizer_Used,Irrigation_Used,Weather_Condition,Days_to_Harvest,Yield_tons_per_hectare
0,West,Sandy,Cotton,897.077239,27.676966,False,True,Cloudy,122,6.555816
1,South,Clay,Rice,992.673282,18.026142,True,True,Rainy,140,8.527341
2,North,Loam,Barley,147.998025,29.794042,False,False,Sunny,106,1.127443
3,North,Sandy,Soybean,986.866331,16.644190,False,True,Rainy,146,6.517573
4,South,Silt,Wheat,730.379174,31.620687,True,True,Cloudy,110,7.248251
...,...,...,...,...,...,...,...,...,...,...
9995,West,Peaty,Maize,634.891339,31.566766,False,False,Rainy,60,4.046993
9996,South,Clay,Maize,518.880035,21.870040,True,False,Cloudy,67,5.105142
9997,East,Chalky,Wheat,364.093250,15.681320,True,False,Cloudy,120,3.774753
9998,East,Silt,Maize,159.269148,28.862575,False,False,Rainy,111,0.783374


In [346]:
yield_df['Soil_Type'].unique()

array(['Sandy', 'Clay', 'Loam', 'Silt', 'Peaty', 'Chalky'], dtype=object)

In [348]:
yield_df.describe()

,Rainfall_mm,Temperature_Celsius,Days_to_Harvest,Yield_tons_per_hectare
count,10000.000000,10000.000000,10000.000000,10000.000000
mean,545.868693,27.487262,104.717400,4.633879
std,259.628158,7.237416,26.095874,1.712395
min,100.105325,15.000276,60.000000,-0.193093
25%,320.898213,21.138845,82.000000,3.374309
50%,542.578396,27.569685,105.000000,4.626792
75%,772.907133,33.851436,128.000000,5.870624
max,999.942517,39.997860,149.000000,9.305258


In [181]:
len(yield_df['Area'].unique())

38

In [307]:
categorical=yield_df.select_dtypes(include=['object']).columns.to_list()

categorical

['Region', 'Soil_Type', 'Crop', 'Weather_Condition']

In [350]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

categorical_columns = yield_df.select_dtypes(include=['object']).columns.tolist()
encoder = OneHotEncoder(sparse_output=False)

one_hot_encoded = encoder.fit_transform(yield_df[categorical_columns])

one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(categorical_columns))

df_encoded = pd.concat([yield_df, one_hot_df], axis=1)

df_encoded = df_encoded.drop(categorical_columns, axis=1)
print(f"Encoded Employee data : \n{df_encoded}")

Encoded Employee data : 
      Rainfall_mm  Temperature_Celsius  Fertilizer_Used  Irrigation_Used  \
0      897.077239            27.676966            False             True   
1      992.673282            18.026142             True             True   
2      147.998025            29.794042            False            False   
3      986.866331            16.644190            False             True   
4      730.379174            31.620687             True             True   
...           ...                  ...              ...              ...   
9995   634.891339            31.566766            False            False   
9996   518.880035            21.870040             True            False   
9997   364.093250            15.681320             True            False   
9998   159.269148            28.862575            False            False   
9999   282.006829            29.045957            False            False   

      Days_to_Harvest  Yield_tons_per_hectare  Region_East  Re

In [352]:
df_encoded

,Rainfall_mm,Temperature_Celsius,Fertilizer_Used,Irrigation_Used,Days_to_Harvest,Yield_tons_per_hectare,Region_East,Region_North,Region_South,Region_West,...,Soil_Type_Silt,Crop_Barley,Crop_Cotton,Crop_Maize,Crop_Rice,Crop_Soybean,Crop_Wheat,Weather_Condition_Cloudy,Weather_Condition_Rainy,Weather_Condition_Sunny
0,897.077239,27.676966,False,True,122,6.555816,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,992.673282,18.026142,True,True,140,8.527341,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,147.998025,29.794042,False,False,106,1.127443,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,986.866331,16.644190,False,True,146,6.517573,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,730.379174,31.620687,True,True,110,7.248251,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,634.891339,31.566766,False,False,60,4.046993,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
9996,518.880035,21.870040,True,False,67,5.105142,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
9997,364.093250,15.681320,True,False,120,3.774753,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
9998,159.269148,28.862575,False,False,111,0.783374,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


### cleaning the data set

In [321]:
cols=yield_df.columns
cols

Index(['Region', 'Soil_Type', 'Crop', 'Rainfall_mm', 'Temperature_Celsius',
       'Fertilizer_Used', 'Irrigation_Used', 'Weather_Condition',
       'Days_to_Harvest', 'Yield_tons_per_hectare'],
      dtype='object')

In [499]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
#df_encoded['Fertilizer_Used'] = df_encoded['Fertilizer_Used'].fit_transform(['Fertilizer_Used'])

# yield_df[categorical[1]]=le.fit_transform(df[categorical[1]])
# yield_df[categorical[2]]=le.fit_transform(df[categorical[2]])
import pandas as pd
df = pd.get_dummies(df_encoded, columns=['Fertilizer_Used','Irrigation_Used'], dtype=int)
df


,Rainfall_mm,Temperature_Celsius,Days_to_Harvest,Yield_tons_per_hectare,Region_East,Region_North,Region_South,Region_West,Soil_Type_Chalky,Soil_Type_Clay,...,Crop_Rice,Crop_Soybean,Crop_Wheat,Weather_Condition_Cloudy,Weather_Condition_Rainy,Weather_Condition_Sunny,Fertilizer_Used_False,Fertilizer_Used_True,Irrigation_Used_False,Irrigation_Used_True
0,897.077239,27.676966,122,6.555816,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1,0,0,1
1,992.673282,18.026142,140,8.527341,0.0,0.0,1.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0,1,0,1
2,147.998025,29.794042,106,1.127443,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1,0,1,0
3,986.866331,16.644190,146,6.517573,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,1,0,0,1
4,730.379174,31.620687,110,7.248251,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,634.891339,31.566766,60,4.046993,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1,0,1,0
9996,518.880035,21.870040,67,5.105142,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0,1,1,0
9997,364.093250,15.681320,120,3.774753,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0,1,1,0
9998,159.269148,28.862575,111,0.783374,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1,0,1,0


In [489]:
df.corrwith(df['Yield_tons_per_hectare'])

Rainfall_mm                 0.765044
Temperature_Celsius         0.096190
Days_to_Harvest            -0.009264
Yield_tons_per_hectare      1.000000
Region_East                -0.010685
Region_North               -0.003777
Region_South                0.005210
Region_West                 0.009171
Soil_Type_Chalky           -0.005045
Soil_Type_Clay              0.007097
Soil_Type_Loam             -0.010281
Soil_Type_Peaty             0.007671
Soil_Type_Sandy            -0.011833
Soil_Type_Silt              0.012441
Crop_Barley                 0.003525
Crop_Cotton                 0.003393
Crop_Maize                  0.006574
Crop_Rice                   0.003927
Crop_Soybean               -0.004565
Crop_Wheat                 -0.012835
Weather_Condition_Cloudy    0.011628
Weather_Condition_Rainy    -0.013172
Weather_Condition_Sunny     0.001497
Fertilizer_Used_False      -0.452582
Fertilizer_Used_True        0.452582
Irrigation_Used_False      -0.362284
Irrigation_Used_True        0.362284
d

In [501]:
X=df.drop(axis=1,columns=['Yield_tons_per_hectare'])



In [503]:
features_to_drop = [
    "Days_to_Harvest",
    "Region_East",
    "Region_North",
    "Region_South",
    "Region_West",
    "Soil_Type_Chalky",
    "Soil_Type_Clay",
    "Soil_Type_Loam",
    "Soil_Type_Peaty",
    "Soil_Type_Sandy",
    "Soil_Type_Silt",
    "Weather_Condition_Cloudy",
    "Weather_Condition_Rainy",
    "Weather_Condition_Sunny"
]

In [505]:
X=X.drop(axis=1,columns=features_to_drop)

In [392]:
y=df['Yield_tons_per_hectare']

In [380]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

num_pipeline = Pipeline([('std_scaler', StandardScaler())])
yield_df = num_pipeline.fit_transform(df)

### Visualizing each datasets

In [384]:
df_encoded

,Rainfall_mm,Temperature_Celsius,Fertilizer_Used,Irrigation_Used,Days_to_Harvest,Yield_tons_per_hectare,Region_East,Region_North,Region_South,Region_West,...,Soil_Type_Silt,Crop_Barley,Crop_Cotton,Crop_Maize,Crop_Rice,Crop_Soybean,Crop_Wheat,Weather_Condition_Cloudy,Weather_Condition_Rainy,Weather_Condition_Sunny
0,897.077239,27.676966,False,True,122,6.555816,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,992.673282,18.026142,True,True,140,8.527341,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,147.998025,29.794042,False,False,106,1.127443,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,986.866331,16.644190,False,True,146,6.517573,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,730.379174,31.620687,True,True,110,7.248251,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,634.891339,31.566766,False,False,60,4.046993,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
9996,518.880035,21.870040,True,False,67,5.105142,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
9997,364.093250,15.681320,True,False,120,3.774753,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
9998,159.269148,28.862575,False,False,111,0.783374,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


0       False
1        True
2       False
3       False
4        True
        ...  
9995    False
9996     True
9997     True
9998    False
9999    False
Name: Fertilizer_Used, Length: 10000, dtype: bool

In [507]:
num_pipeline = Pipeline([('std_scaler', StandardScaler())])
X = num_pipeline.fit_transform(X)
X

array([[ 1.35280438,  0.02621295, -0.44372074, ..., -1.00220243,
        -1.0034058 ,  1.0034058 ],
       [ 1.72102646, -1.30731644, -0.44372074, ...,  0.99780241,
        -1.0034058 ,  1.0034058 ],
       [-1.5325401 ,  0.31874581,  2.25366975, ..., -1.00220243,
         0.99660576, -0.99660576],
       ...,
       [-0.70017264, -1.63131874, -0.44372074, ...,  0.99780241,
         0.99660576, -0.99660576],
       [-1.48912537,  0.19003765, -0.44372074, ..., -1.00220243,
         0.99660576, -0.99660576],
       [-1.01635763,  0.215377  , -0.44372074, ..., -1.00220243,
         0.99660576, -0.99660576]])

In [509]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)


### GradientBoosting Algorithm

In [543]:
from sklearn.ensemble import GradientBoostingRegressor
boost_model=GradientBoostingRegressor().fit(X_train,y_train)

In [545]:
pred=boost_model.predict(X_test)

In [547]:
from sklearn.metrics import mean_absolute_error,mean_squared_error

In [549]:
mean_absolute_error(y_test,pred)

0.41438228454345855

In [551]:
mean_squared_error(y_test,pred)

0.26613320114567274

In [553]:
from sklearn.neural_network import MLPRegressor

In [555]:
mlp_model=MLPRegressor().fit(X_train,y_train)
pred=mlp_model.predict(X_test)

In [556]:
mean_absolute_error(y_test,pred)

0.4096483354282805

In [559]:
from sklearn.linear_model import LinearRegression
linear_model=LinearRegression().fit(X_train,y_train)

In [561]:
pred=linear_model.predict(X_test)

In [563]:
mean_absolute_error(y_test,pred)

0.4079090776758728

In [567]:
import joblib
# Save the trained model
joblib.dump(boost_model, 'crop_yield_model.pkl')
print("Model saved successfully!")

Model saved successfully!
